<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/lab04_team5/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 04: Analysis of Movies Data Frame Part II

#### Team 5:
##### Luis Miguel Fontes Cardona
##### Gabriel Alejandro Olvera González
##### Marian Montserrat Sedano Paz


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,267 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 https://r2u.stat.ill

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Movies-Lab04").getOrCreate()

df_movies = spark.read \
            .option("inferSchema", "true") \
            .json("/content/movies.json")

In [23]:
df_movies.printSchema()

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)



In [24]:
df_movies.show(n=7)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land Girls|        NULL|  146083|         146083|


Calculate Profit Margin: Compute the profit margin for each movie
by subtracting the Production Budget from the Worldwide Gross
and then dividing by the Production Budget


In [25]:
df = df_movies.withColumn('Profit_Margin', (df_movies['Worldwide_Gross'] - df_movies['Production_Budget']) / df_movies['Production_Budget'])

df.show(n=7)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|      Profit_Margin|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|

In [26]:
# new DataFrame
df.printSchema()

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)
 |-- Profit_Margin: double (nullable = true)



Determine if a Movie is a Box Office Hit: Create a column
Box Office Hit that categorizes movies as ”Hit” if
Worldwide Gross is greater than twice the Production Budget,
and ”Flop” otherwise.

In [27]:
from pyspark.sql.functions import when
df = df_movies.withColumn('Box_Office_Hit',
                          when(df['Worldwide_Gross'] > (df['Production_Budget'] * 2), "Hit").otherwise("Flop"))
df.show(n=7)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+--------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Box_Office_Hit|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+--------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land 

Convert Release Date to a Standard Format: Convert the
Release Date from DD-MMM-YY to a DateType column and
extract the year in a new column.


In [29]:
from pyspark.sql.functions import col, to_date, year, when, expr

# Convert Release_Date from string to DateType and extract the year
df = df_movies.withColumn(
    'Release_DateType',
    when(col('Release_Date').rlike("^\d{1,2}-[a-zA-Z]{3}-\d{2}$"),
         to_date(col('Release_Date'), 'd-MMM-yy'))
    .otherwise(None)  # Handle unexpected formats with None
)

# Adjust for years greater than 2023 by subtracting 100 years
df = df.withColumn(
    'Release_DateType',
    when(year(col('Release_DateType')) > 2023,
         expr("add_months(Release_DateType, -1200)"))  # Subtract 100 years
    .otherwise(col('Release_DateType'))
)

# Extract the year from the Release_DateType
df = df.withColumn('Release_Year', year(col('Release_DateType')))

df.show(n=7)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------+------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Release_DateType|Release_Year|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------+------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|       

Create a new column: IMDB Rating Category to categorize movies
based on their IMDB Rating: ”High” if the rating is 7.0 or above,
”Medium” if between 5.0 and 6.9, and ”Low” if below 5.0.

In [30]:
# Create the IMDB_Rating_Category column with the categories according to the IMDB rating.

df = df.withColumn('IMDB_Rating_Category',
                  when(col('IMDB_Rating') >= 7.0, 'High')
                  .when((col('IMDB_Rating') >= 5.0) & (col('IMDB_Rating') < 7.0), 'Medium')
                  .otherwise('Low')
)

df.show(n=7)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------+------------+--------------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Release_DateType|Release_Year|IMDB_Rating_Category|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------+------------+--------------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NUL

Calculate Average IMDB Rating for Each Distributor: Calculate the
average IMDB Rating for each Distributor and create a new
DataFrame with this information.

In [31]:
from pyspark.sql.functions import avg

# Calculate the average IMDB_Rating for each Distributor
df_avg_rating = df.groupBy('Distributor') \
                  .agg(avg('IMDB_Rating').alias('Average_IMDB_Rating'))

# new DataFrame
df_avg_rating.show(n=7)

+--------------------+-------------------+
|         Distributor|Average_IMDB_Rating|
+--------------------+-------------------+
|Oscilloscope Pict...|                6.2|
|               Savoy|  6.533333333333334|
|             Embassy|               NULL|
|         Fader Films|                6.5|
|       October Films|                6.4|
|              Strand|  6.322222222222223|
|             Trimark|  5.085714285714286|
+--------------------+-------------------+
only showing top 7 rows

